In [21]:
import torch
from utils import *
from scipy.optimize import minimize, least_squares
printify = lambda f: lambda *args, **kwargs: print(f(*args, **kwargs), end='\n' + '-'*50 + '\n')

In [44]:
pminimize = printify(minimize)

pminimize(lambda x: (x[0] - torch.pi)**2 + (x[1] + torch.pi/2)**2, [0., 0.])
pminimize(lambda x: ((x[0]-2) + (x[1]-3))**2, [0., 0.])
pminimize(lambda x: ((x[0]-2) + (x[1]-3))**2, [-2., 0.], bounds=[(-np.inf,-4), (-np.inf,np.inf)])
pminimize(lambda x: ((x[0]-2) + (x[1]-3))**2, [-5., 0.], bounds=[(-np.inf,-4), (-np.inf,np.inf)])
pminimize(lambda x: ((x[0]-2) + (x[1]-3))**2, [-5., 0.], bounds=[(-4,-4), (-np.inf,np.inf)])

      fun: 2.7605759509528782e-14
 hess_inv: array([[0.60000004, 0.20000003],
       [0.20000003, 0.89999997]])
      jac: array([1.57835371e-07, 3.14889244e-07])
  message: 'Optimization terminated successfully.'
     nfev: 12
      nit: 3
     njev: 4
   status: 0
  success: True
        x: array([ 3.14159273, -1.57079618])
--------------------------------------------------
      fun: 3.542141690395744e-17
 hess_inv: array([[ 0.625, -0.375],
       [-0.375,  0.625]])
      jac: array([2.99798231e-09, 2.99798231e-09])
  message: 'Optimization terminated successfully.'
     nfev: 12
      nit: 3
     njev: 4
   status: 0
  success: True
        x: array([2.5, 2.5])
--------------------------------------------------
      fun: 2.500003966417107e-17
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.00000079e-08, -7.10542736e-15])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 12
      nit: 3
     njev: 4
   status: 0
  success: True
  

In [43]:
pleast_squares = printify(least_squares)

pleast_squares(lambda x: (x[0] - torch.pi)**2 + (x[1] + torch.pi/2)**2, [0., 0.])
pleast_squares(lambda x: ((x[0]-2) + (x[1]-3))**2, [0., 0.])
pleast_squares(lambda x: ((x[0]-2) + (x[1]-3))**2, [-4., 0.], bounds= ([-np.inf, -np.inf], [-4, np.inf]))
pleast_squares(lambda x: ((x[0]-2) + (x[1]-3))**2, [-4., 0.], bounds= ([-4 - 1e-10, -np.inf], [-4., np.inf]))

 active_mask: array([0., 0.])
        cost: 1.2783752744478993e-12
         fun: array([1.59898422e-06])
        grad: array([ 3.61701446e-09, -1.80850731e-09])
         jac: array([[ 0.00226207, -0.00113104]])
     message: '`gtol` termination condition is satisfied.'
        nfev: 17
        njev: 10
  optimality: 3.6170144554807505e-09
      status: 1
     success: True
           x: array([ 3.14272367, -1.57136183])
--------------------------------------------------
 active_mask: array([0., 0.])
        cost: 2.5731570692902734e-13
         fun: array([7.17378153e-07])
        grad: array([1.21523919e-09, 1.21523919e-09])
         jac: array([[0.001694, 0.001694]])
     message: '`gtol` termination condition is satisfied.'
        nfev: 16
        njev: 11
  optimality: 1.2152391925996551e-09
      status: 1
     success: True
           x: array([2.50042349, 2.50042349])
--------------------------------------------------
 active_mask: array([0, 0])
        cost: 3.891221023383139e

#### a) Градиент